In [1]:
single_file = "/data3/epyc/data3/hipscat/raw/dp02/objectTable_tract_3447_DC2_2_2i_runs_DP0_2_v23_0_1_PREOPS-905_step3_9_20220302T181001Z.parq"



In [8]:
import pyarrow.parquet as pq

md = pq.read_metadata(single_file)
md.schema.to_arrow_schema().pandas_metadata

{'index_columns': ['objectId'],
 'column_indexes': [{'name': 'column',
   'field_name': 'column',
   'pandas_type': 'unicode',
   'numpy_type': 'object',
   'metadata': {'encoding': 'UTF-8'}}],
 'columns': [{'name': 'detect_isIsolated',
   'field_name': 'detect_isIsolated',
   'pandas_type': 'bool',
   'numpy_type': 'bool',
   'metadata': None},
  {'name': 'x',
   'field_name': 'x',
   'pandas_type': 'float64',
   'numpy_type': 'float64',
   'metadata': None},
  {'name': 'refBand',
   'field_name': 'refBand',
   'pandas_type': 'unicode',
   'numpy_type': 'object',
   'metadata': None},
  {'name': 'footprintArea',
   'field_name': 'footprintArea',
   'pandas_type': 'int32',
   'numpy_type': 'int32',
   'metadata': None},
  {'name': 'coord_dec',
   'field_name': 'coord_dec',
   'pandas_type': 'float64',
   'numpy_type': 'float64',
   'metadata': None},
  {'name': 'shape_flag',
   'field_name': 'shape_flag',
   'pandas_type': 'bool',
   'numpy_type': 'bool',
   'metadata': None},
  {'name

In [11]:
macauff_path = "/data3/epyc/data3/hipscat/test_catalogs/macauff_association/_common_metadata"

md = pq.read_metadata(macauff_path)
md.schema.to_arrow_schema().metadata

{b'pandas': b'{"index_columns": [], "column_indexes": [], "columns": [{"name": "gaia_source_id", "field_name": "gaia_source_id", "pandas_type": "int64", "numpy_type": "int64", "metadata": null}, {"name": "gaia_ra", "field_name": "gaia_ra", "pandas_type": "float64", "numpy_type": "float64", "metadata": null}, {"name": "gaia_dec", "field_name": "gaia_dec", "pandas_type": "float64", "numpy_type": "float64", "metadata": null}, {"name": "BP", "field_name": "BP", "pandas_type": "float64", "numpy_type": "float64", "metadata": null}, {"name": "G", "field_name": "G", "pandas_type": "float64", "numpy_type": "float64", "metadata": null}, {"name": "RP", "field_name": "RP", "pandas_type": "float64", "numpy_type": "float64", "metadata": null}, {"name": "catwise_name", "field_name": "catwise_name", "pandas_type": "unicode", "numpy_type": "object", "metadata": null}, {"name": "catwise_ra", "field_name": "catwise_ra", "pandas_type": "float64", "numpy_type": "float64", "metadata": null}, {"name": "catwi

In [23]:
import pandas as pd
import pyarrow as pa

input_csv = "/astro/users/mmd11/git/scripts/epyc/dp0/metadata.csv"

input_df = pd.read_csv(input_csv, dtype="str", keep_default_na=False)

fields = []

def _construct_field(name, type_str, metadata_dict):
    if type_str in ("string", "char"):
        pa_type = pa.string()
    elif type_str in ("float", "double"):
        pa_type = pa.float64()
    elif type_str in ("int", "long"):
        pa_type = pa.int64()
    elif type_str in ("boolean"):
        pa_type = pa.bool_()
    else:
        raise ValueError(f"unhandled type string {type_str}")
#     print(name, pa_type, metadata_dict)
    return pa.field(name, pa_type, metadata=metadata_dict)


for _, row in input_df.iterrows():
#     print(row)
    name = row["Column Name"]
    description = row["Description"]
    type_str = row["Data Type"]
    metadata_dict = {
        "Description": description,
        "Unit":row["Unit"] or "",
        "UCD" : row["UCD"],
    }

    fields.append(_construct_field(name, type_str, metadata_dict))

schema = pa.schema(fields)
pq.write_table(schema.empty_table(), where="/astro/users/mmd11/git/scripts/epyc/dp0/metadata.parquet")


In [24]:
import os
import pyarrow.parquet as pq

sample_parquet_file = "/data3/epyc/data3/hipscat/raw/dp02/objectTable_tract_3447_DC2_2_2i_runs_DP0_2_v23_0_1_PREOPS-905_step3_9_20220302T181001Z.parq"

sample_file_size = os.path.getsize(sample_parquet_file)
parquet_file = pq.ParquetFile(sample_parquet_file)
num_rows = parquet_file.metadata.num_rows

## 100MB
ideal_file_small = 100 * 1024 * 1024
## 800MB
ideal_file_large = 800 * 1024 * 1024

threshold_small = ideal_file_small / sample_file_size * num_rows
threshold_large = ideal_file_large / sample_file_size * num_rows

print(f"threshold between {int(threshold_small):_} and {int(threshold_large):_}")

threshold between 27_092 and 216_739
